for the moment, uncomment out the parts you need to run. if first time running, run all

### 0 : importing packages

In [1]:
import json
from pathlib import Path, PurePath 
import glob

import betfairlightweight
from betfairlightweight import filters

import datetime

import pandas as pd
import numpy as np

from bz2 import BZ2File 

from betfairlightweight import StreamListener
from betfairlightweight.streaming.stream import MarketStream

import data_utils
from data_utils import *

### 1 : 'logging in'

In [2]:
project_dir = Path.cwd().parents[1]
logins_dir = project_dir / 'api_logins.json'

with open(logins_dir) as f:
    login_dict =  json.load(f)
    
trading = betfairlightweight.APIClient(username=login_dict['my_username'],
                                       password=login_dict['my_password'],
                                       app_key=login_dict['my_app_key'],
                                       certs=login_dict['certs_path'])

trading.login()

<LoginResource>

### 2 : retrieving api data list

In [3]:
# # returns list of 'data dictionaries'
# data_dicts = trading.historic.get_my_data()

# # calculate range of dates for advanced data
# adv_range = [d['forDate'] for d in data_dicts if d['plan'] == 'Advanced Plan']

# # find min date for adv_data
# adv_min_date = datetime.datetime.strptime(min(adv_range), '%Y-%m-%dT%H:%M:%S')

# # find max data for adv data
# def last_day_of_month(any_day):
#     next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  
#     return next_month - datetime.timedelta(days=next_month.day)

# adv_max_temp = datetime.datetime.strptime(max(adv_range), '%Y-%m-%dT%H:%M:%S')
# adv_max_date = last_day_of_month(adv_max_temp) 

In [4]:
# # list files within advanced data range (GB Data)
# adv_file_list = trading.historic.get_file_list(
#     "Horse Racing",
#     "Advanced Plan",
#     from_day=adv_min_date.day,
#     from_month=adv_min_date.day,
#     from_year=adv_min_date.year,
#     to_day=adv_max_date.day,
#     to_month=adv_max_date.month,
#     to_year=adv_max_date.year,
#     market_types_collection=["WIN"],
#     countries_collection=["GB"],
#     file_type_collection=["M"],
# )
# print("No. items :", len(adv_file_list))

### 3 : downloading data
DON'T interrupt this when downloading or file may come corrupted and cause issues when processing

In [5]:
# # where to store our advanced data
raw_dir = project_dir / 'data' / 'raw' / 'api' / 'advanced'

# # check if files have been downloaded already
# # adv_all_files = [Path(f).name for f in adv_file_list] # all files to download
# adv_downloaded_files = [Path(f).name for f in raw_dir.glob("*.bz2")] # all files downloaded

# # download files we dont have and writing uncompressed versions
# for file in adv_file_list: # remove index for all files
#     if Path(file).name not in adv_downloaded_files: 
#         print(file)
#         download = trading.historic.download_file(file_path = file, store_directory = raw_dir)
#         print(download)

### 4 : processing data

decompressing and writing

In [6]:
# proceesing bz2 to text (try/except for thos files corrupted when downloaded, not many)
# adv_extfile_dirs = []

# for file in glob.glob(str(adv_dir) + '/*.bz2'): # change this?
#     try:
#         zipfile = BZ2File(file) # open the file
#         data = zipfile.read() # get the decompressed data
#         newfilepath = file.split('.bz2')[0] # removing the extension and saving without a filetype
#         open(newfilepath, 'wb').write(data) # write an uncompressed file
#         adv_extfile_dirs.append(newfilepath)
#         zipfile.close()
#     except OSError:
#         print("File not processed : ", file)
#         pass

In [7]:
# add code in to then redownload these files that are corrupted??

### 5 : 'streaming' / writing processed data

In [8]:
processed_files = glob.glob(str(raw_dir) + '/*[!.bz2]')

In [11]:
datadict = {'Time': [],
       'MarketId' : [],
       'Status' : [],
       'Inplay' : [], 
       'SelectionId' : [],
       'LastPriceTraded' : [],
       'TotalMatched' : [],
       'BSP' : [],
       'AdjFactor' :  [],
       'RunnerStatus' : [],
       'MktTotalMatched' : [],
       'RaceInfo' : [],
       'Venue' : [],
       'BackSize': [],
       'BackPrice': [],
       'LayPrice' : [],
       'LaySize' : []
}

d = datadict

In [12]:
class HistoricalStream(MarketStream):
    # create custom listener and stream

    def _init_(self, listener):
        super(HistoricalStream, self)._init_(listener)

    def on_process(self, market_books):
        for market_book in market_books:
            for runner in market_book.runners:

                datadict['Time'].append(market_book.publish_time)
                datadict['MarketId'].append(float(market_book.market_id))
                datadict['Status'].append(market_book.status)
                datadict['Inplay'].append(market_book.inplay)
                datadict['SelectionId'].append(runner.selection_id)
                datadict['LastPriceTraded'].append(runner.last_price_traded)
                datadict['TotalMatched'].append(runner.total_matched)
                datadict['BSP'].append(runner.sp.actual_sp)
                datadict['AdjFactor'].append(runner.adjustment_factor)
                datadict['RunnerStatus'].append(runner.status)
                datadict['MktTotalMatched'].append(market_book.total_matched)
                datadict['RaceInfo'].append(market_book.market_definition.name)
                datadict['Venue'].append(market_book.market_definition.venue)
                
                atb_size = [x.size for x in runner.ex.available_to_back]
                datadict['BackSize'].append(atb_size)
                atb_price = [x.price for x in runner.ex.available_to_back]
                datadict['BackPrice'].append(atb_price)   
                atl_price = [x.price for x in runner.ex.available_to_lay]
                datadict['LayPrice'].append(atl_price)
                atl_size = [x.size for x in runner.ex.available_to_lay]
                datadict['LaySize'].append(atl_size)


class HistoricalListener(StreamListener):
    def _add_stream(self, unique_id, stream_type):
        if stream_type == "marketSubscription":
            return HistoricalStream(self)

In [13]:
listener = HistoricalListener(max_latency=None)

In [14]:
final_df = pd.DataFrame()
interim_dir = project_dir / 'data' / 'interim' / 'api' / 'advanced'

T_PRE = 60
T_POST = 15


for file in processed_files:
    try:
        datadict = {'Time': [],
           'MarketId' : [],
           'Status' : [],
           'Inplay' : [], 
           'SelectionId' : [],
           'LastPriceTraded' : [],
           'TotalMatched' : [],
           'BSP' : [],
           'AdjFactor' :  [],
           'RunnerStatus' : [],
           'MktTotalMatched' : [],
           'RaceInfo' : [],
           'Venue' : [],
           'BackSize': [],
           'BackPrice': [],
           'LayPrice' : [],
           'LaySize' : []
        }


        stream = trading.streaming.create_historical_stream(directory=file, listener=listener)
        stream.start() 
        stream.stop()

        print(str(file) + " stream completed.")

        # create df
        df = dict_to_df(datadict)

        df = pre_filter_races(df);
        df = extract_race_info(df);
        df = create_timedif(df);
        df = filter_timedif(df);
        df = create_time_bins(df, T_PRE, T_POST);
        df = runner_groupby(df, T_PRE, T_POST)

        # write / read from here

        final_df = pd.concat([final_df, df])
        print('Rows added : ', len(df.index), 'Total rows : ', len(final_df.index))
        del df
        

    except:
        print("File not processed : ", file)
        pass

/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169028429 stream completed.
Rows added :  8 Total rows :  8
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168013705 stream completed.
Rows added :  14 Total rows :  22
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169341238 stream completed.
Rows added :  12 Total rows :  34
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168473844 stream completed.
Rows added :  10 Total rows :  44
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167971949 stream completed.
Rows added :  9 Total rows :  53
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169057140 stream completed.
Rows added :  14 Total rows :  67
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169866063 stream completed.
R

Rows added :  9 Total rows :  434
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167074916 stream completed.
Rows added :  8 Total rows :  442
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167748193 stream completed.
Rows added :  9 Total rows :  451
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169434613 stream completed.
Rows added :  10 Total rows :  461
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169668051 stream completed.
Rows added :  0 Total rows :  461
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168799528 stream completed.
Rows added :  8 Total rows :  469
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167206255 stream completed.
Rows added :  0 Total rows :  469
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/ad

Rows added :  7 Total rows :  827
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.170118750 stream completed.
Rows added :  8 Total rows :  835
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168672433 stream completed.
Rows added :  5 Total rows :  840
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169500951 stream completed.
Rows added :  8 Total rows :  848
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169309187 stream completed.
Rows added :  4 Total rows :  852
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169920596 stream completed.
Rows added :  17 Total rows :  869
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169057114 stream completed.
Rows added :  12 Total rows :  881
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/a

/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167831510 stream completed.
Rows added :  11 Total rows :  1298
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167787598 stream completed.
Rows added :  16 Total rows :  1314
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168203090 stream completed.
Rows added :  10 Total rows :  1324
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169668066 stream completed.
Rows added :  0 Total rows :  1324
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167074921 stream completed.
Rows added :  7 Total rows :  1331
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169309119 stream completed.
Rows added :  6 Total rows :  1337
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168298585 stream 

/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168671423 stream completed.
Rows added :  5 Total rows :  1757
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169309144 stream completed.
Rows added :  6 Total rows :  1763
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167390321 stream completed.
Rows added :  9 Total rows :  1772
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169101610 stream completed.
Rows added :  0 Total rows :  1772
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168336924 stream completed.
Rows added :  10 Total rows :  1782
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167106104 stream completed.
Rows added :  9 Total rows :  1791
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167486111 stream co

...

Rows added :  6 Total rows :  11202
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169434748 stream completed.
Rows added :  0 Total rows :  11202
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168754718 stream completed.
Rows added :  7 Total rows :  11209
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169434783 stream completed.
Rows added :  0 Total rows :  11209
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167206365 stream completed.
Rows added :  8 Total rows :  11217
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167923803 stream completed.
Rows added :  13 Total rows :  11230
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.170188304 stream completed.
Rows added :  0 Total rows :  11230
/Users/tombardrick/Documents/projects/betfair/betfair_project/d

Rows added :  10 Total rows :  11653
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168298595 stream completed.
Rows added :  9 Total rows :  11662
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168392129 stream completed.
Rows added :  10 Total rows :  11672
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167074931 stream completed.
Rows added :  6 Total rows :  11678
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167829706 stream completed.
Rows added :  8 Total rows :  11686
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168755319 stream completed.
Rows added :  6 Total rows :  11692
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168203080 stream completed.
Rows added :  12 Total rows :  11704
/Users/tombardrick/Documents/projects/betfair/betfair_project

Rows added :  10 Total rows :  12163
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168474023 stream completed.
Rows added :  16 Total rows :  12179
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169155729 stream completed.
Rows added :  5 Total rows :  12184
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167134930 stream completed.
Rows added :  10 Total rows :  12194
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.166897838 stream completed.
Rows added :  7 Total rows :  12201
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168575484 stream completed.
Rows added :  7 Total rows :  12208
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168203075 stream completed.
Rows added :  5 Total rows :  12213
/Users/tombardrick/Documents/projects/betfair/betfair_project

Rows added :  11 Total rows :  12585
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168799553 stream completed.
Rows added :  13 Total rows :  12598
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168014760 stream completed.
Rows added :  8 Total rows :  12606
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.166942610 stream completed.
Rows added :  8 Total rows :  12614
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169760563 stream completed.
Rows added :  12 Total rows :  12626
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168208898 stream completed.
Rows added :  5 Total rows :  12631
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.169561940 stream completed.
Rows added :  7 Total rows :  12638
/Users/tombardrick/Documents/projects/betfair/betfair_project

Rows added :  5 Total rows :  13059
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.170027024 stream completed.
Rows added :  14 Total rows :  13073
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.167451017 stream completed.
Rows added :  0 Total rows :  13073
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/advanced/1.168575424 stream completed.
Rows added :  0 Total rows :  13073


In [15]:
final_df.head()

,SelectionId,MarketId,Venue,Distance,RaceType,BSP,NoRunners,BS:T-60,BS:T-59,BS:T-58,...,LS:T+5,LS:T+6,LS:T+7,LS:T+8,LS:T+9,LS:T+10,LS:T+11,LS:T+12,LS:T+13,LS:T+14
0,11986132,1.169028429,Huntingdon,20.0,Chase,8.33,9,16.43,24.51,26.57,...,10.08,11.15,5.44,7.09,14.16,19.53,3.12,3.31,0.68,0.68
1,16800725,1.169028429,Huntingdon,20.0,Chase,3.68,9,15.43,25.74,57.82,...,29.87,221.22,43.23,43.10,13.53,26.15,13.60,74.30,419.52,23082.10
2,20968322,1.169028429,Huntingdon,20.0,Chase,14.96,9,9.87,9.25,9.15,...,37.32,6.83,4.85,11.23,16.00,5.68,40.25,12.51,10.42,13.17
3,22023486,1.169028429,Huntingdon,20.0,Chase,4.25,9,84.38,64.49,58.01,...,11.67,2.02,2.02,2.02,2.02,2.02,2.02,2.02,2.02,2.02
4,24496216,1.169028429,Huntingdon,20.0,Chase,6.60,9,10.64,10.11,7.91,...,34.27,54.72,11.85,17.99,48.21,17.28,38.29,6.96,4.37,4.37


In [18]:
final_df.shape

(13073, 307)

### 6 : writing out data

In [16]:
processed_dir = project_dir / 'data' / 'processed' / 'api' / 'advanced'
final_df.to_csv(str(processed_dir) + '/adv_data.csv')

In [ ]:
# To-Do
# use multiindexing for columns for time points / prices/ sizes in future?
# move .py version to src/data/historic
# investigate nulls befre removal in filtering
# define 'hard variables' with capitals e.g. directories

# add doc strings to each function e.g. qcut
# include irish/us racing also

# add in other variables like no.runners, country, date

# add in conditions to make script 'run beackawards' e.g.
# - if market name not in final_df file then stream - and add it in - call this updates?

# then wat to do with unprocessed files, redownload these?